In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import wandb
import time
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [2]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar


--2025-05-20 11:41:32--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.207, 64.233.188.207, 173.194.174.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  30.0MB/s    in 65s     

2025-05-20 11:42:37 (29.4 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [3]:
import wandb

wandb.login(key="93b4881869bab13360839595daa56e51dd0405df")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivam-da24m018 (shivam-da24m018-iitmaana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
!tar -xf dakshina_dataset_v1.0.tar

In [5]:
# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [6]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [7]:
# Define the Encoder class
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, cell_type='RNN', num_layers=1, dropout=0.0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embed_size)
        self.cell_type = cell_type.upper()
        
        rnn_class = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[self.cell_type]
        self.rnn = rnn_class(
            input_size=embed_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        if self.cell_type == 'LSTM':
            outputs, (hidden, cell) = self.rnn(embedded)
            return outputs, (hidden, cell)
        else:
            outputs, hidden = self.rnn(embedded)
            return outputs, hidden


In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, cell_type='RNN', num_layers=1, dropout=0.0):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        self.cell_type = cell_type.upper()
        
        rnn_class = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[self.cell_type]
        self.rnn = rnn_class(
            input_size=embed_size,  # Without attention, input is just the embedding
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.out = nn.Linear(hidden_size, output_size)  # Output layer without context vector

    def forward(self, input_char, hidden):
        embedded = self.embedding(input_char).unsqueeze(1)  # (batch_size, 1, embed_size)
        embedded = self.dropout(embedded)
        
        if self.cell_type == 'LSTM':
            output, (hidden, cell) = self.rnn(embedded, hidden)
            output = output.squeeze(1)  # (batch_size, hidden_size)
            output = self.out(output)  # (batch_size, output_size)
            return output, (hidden, cell)
        else:
            output, hidden = self.rnn(embedded, hidden)
            output = output.squeeze(1)
            output = self.out(output)
            return output, hidden


In [9]:
# Define the Seq2Seq model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.out.out_features
        
        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
        
        # Encode the source sequence
        _, hidden = self.encoder(source)
        
        # First input to the decoder is the <SOS> token
        decoder_input = target[:, 0]
        
        # Handle differing encoder/decoder layers
        if self.encoder.cell_type == 'LSTM':
            hidden, cell = hidden
            if self.encoder.num_layers != self.decoder.num_layers:
                factor = self.decoder.num_layers // self.encoder.num_layers + 1
                hidden = hidden.repeat(factor, 1, 1)[:self.decoder.num_layers]
                cell = cell.repeat(factor, 1, 1)[:self.decoder.num_layers]
            hidden = (hidden, cell)
        else:
            if self.encoder.num_layers != self.decoder.num_layers:
                factor = self.decoder.num_layers // self.encoder.num_layers + 1
                hidden = hidden.repeat(factor, 1, 1)[:self.decoder.num_layers]
        
        # Decode one step at a time
        for t in range(1, target_len):
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t, :] = output
            
            # Teacher forcing: use real target as next input with probability teacher_forcing_ratio
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            decoder_input = target[:, t] if teacher_force else top1

        return outputs


In [10]:
# Define data loading and preprocessing functions
def load_data(file_path):
    """Load data from the file path"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                latin = parts[1]
                devanagari = parts[0]
                data.append((latin, devanagari))
    return data

def create_vocab(data):
    """Create vocabulary from data"""
    latin_chars = set()
    devanagari_chars = set()
    devanagari_chars = set()
    for _, devanagari in data:
        devanagari_chars.update(devanagari)
    for latin, devanagari in data:
        latin_chars.update(latin)
        #devanagari_chars.update(devanagari)
    
    print("Sample Devanagari chars:", sorted(devanagari_chars)[:10])
    print("Sample Latin chars:", sorted(latin_chars)[:10])
    # Add special tokens
    latin_chars.add('<pad>')
    latin_chars.add('<sos>')
    latin_chars.add('<eos>')
    
    devanagari_chars.add('<pad>')
    devanagari_chars.add('<sos>')
    devanagari_chars.add('<eos>')
    
    # Create dictionaries
    latin_to_idx = {char: idx for idx, char in enumerate(sorted(latin_chars))}
    idx_to_latin = {idx: char for char, idx in latin_to_idx.items()}
    
    devanagari_to_idx = {char: idx for idx, char in enumerate(sorted(devanagari_chars))}
    idx_to_devanagari = {idx: char for char, idx in devanagari_to_idx.items()}
    
    return latin_to_idx, idx_to_latin, devanagari_to_idx, idx_to_devanagari

def encode_data(data, latin_to_idx, devanagari_to_idx, max_latin_len=None, max_devanagari_len=None):
    """Encode data using vocabulary mappings"""
    if max_latin_len is None:
        max_latin_len = max(len(latin) for latin, _ in data) + 2  # +2 for <sos> and <eos>
    if max_devanagari_len is None:
        max_devanagari_len = max(len(devanagari) for _, devanagari in data) + 2
    
    latin_encoded = []
    devanagari_encoded = []
    
    for latin, devanagari in data:
        # Encode Latin
        latin_seq = [latin_to_idx['<sos>']]
        latin_seq.extend(latin_to_idx[char] for char in latin)
        latin_seq.append(latin_to_idx['<eos>'])
        # Pad Latin
        latin_seq.extend([latin_to_idx['<pad>']] * (max_latin_len - len(latin_seq)))
        latin_encoded.append(latin_seq)
        
        # Encode Devanagari
        devanagari_seq = [devanagari_to_idx['<sos>']]
        devanagari_seq.extend(devanagari_to_idx[char] for char in devanagari)
        devanagari_seq.append(devanagari_to_idx['<eos>'])
        # Pad Devanagari
        devanagari_seq.extend([devanagari_to_idx['<pad>']] * (max_devanagari_len - len(devanagari_seq)))
        devanagari_encoded.append(devanagari_seq)
    
    return torch.tensor(latin_encoded), torch.tensor(devanagari_encoded), max_latin_len, max_devanagari_len


In [11]:
# Define the TransliterationDataset
class TransliterationDataset(Dataset):
    def __init__(self, source_tensor, target_tensor):
        self.source_tensor = source_tensor
        self.target_tensor = target_tensor
        
    def __len__(self):
        return len(self.source_tensor)
    
    def __getitem__(self, idx):
        return self.source_tensor[idx], self.target_tensor[idx]


In [12]:
# Define training and evaluation functions
def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio):
    model.train()
    epoch_loss = 0
    
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg, teacher_forcing_ratio)
        
        # Reshape output and target for loss calculation
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            
            output = model(src, trg, 0)  # No teacher forcing during evaluation
            
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

def calculate_accuracy(model, iterator, devanagari_to_idx, idx_to_devanagari):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    
    with torch.no_grad():
        for src, trg in iterator:
            src, trg = src.to(device), trg.to(device)
            batch_size = src.shape[0]
            
            # Encode the source sequence
            _, hidden = model.encoder(src)
            
            # Start with <sos> token
            decoder_input = torch.tensor([devanagari_to_idx['<sos>']] * batch_size).to(device)
            
            # Store predictions
            trg_len = trg.shape[1]
            predictions_batch = torch.zeros(batch_size, trg_len).to(device)
            predictions_batch[:, 0] = decoder_input
            
            for t in range(1, trg_len):
                output, hidden = model.decoder(decoder_input, hidden)
                top1 = output.argmax(1)
                predictions_batch[:, t] = top1
                decoder_input = top1
            
            # Compare predictions with targets
            for i in range(batch_size):
                pred_seq = []
                target_seq = []
                
                for j in range(1, trg_len):  # Skip <sos>
                    pred_idx = predictions_batch[i, j].item()
                    target_idx = trg[i, j].item()
                    
                    if pred_idx == devanagari_to_idx['<eos>'] or pred_idx == devanagari_to_idx['<pad>']:
                        break
                    if target_idx == devanagari_to_idx['<eos>'] or target_idx == devanagari_to_idx['<pad>']:
                        break
                    
                    pred_seq.append(idx_to_devanagari[pred_idx])
                    target_seq.append(idx_to_devanagari[target_idx])
                
                pred_word = ''.join(pred_seq)
                target_word = ''.join(target_seq)
                
                predictions.append((pred_word, target_word))
                
                if pred_word == target_word:
                    correct += 1
                total += 1
    
    accuracy = correct / total
    return accuracy, predictions


In [13]:
# Define function to calculate model complexity
def calculate_model_complexity(m, k, T, V):
    """
    Calculate model complexity
    m: embedding size
    k: hidden state size
    T: sequence length
    V: vocabulary size
    """
    # Embedding layer parameters
    embedding_params = 2 * V * m  # Source and target embeddings
    
    # RNN parameters (for both encoder and decoder)
    encoder_rnn_params = m * k + k * k + k  # Input-to-hidden, hidden-to-hidden, bias
    decoder_rnn_params = m * k + k * k + k
    
    # Output layer parameters
    output_params = k * V + V  # Hidden-to-output, bias
    
    # Total parameters
    total_params = embedding_params + encoder_rnn_params + decoder_rnn_params + output_params
    
    # Computations
    # Encoder computations
    encoder_comp = T * (m * k + k * k)
    
    # Decoder computations
    decoder_comp = T * (m * k + k * k + k * V)
    
    # Total computations
    total_comp = encoder_comp + decoder_comp
    
    return {
        'embedding_params': embedding_params,
        'encoder_rnn_params': encoder_rnn_params,
        'decoder_rnn_params': decoder_rnn_params,
        'output_params': output_params,
        'total_params': total_params,
        'encoder_comp': encoder_comp,
        'decoder_comp': decoder_comp,
        'total_comp': total_comp
    }


In [14]:
# Define function to initialize model with given hyperparameters
def init_model(config, input_size, output_size):
    encoder = EncoderRNN(
        input_size=input_size,
        embed_size=config.embed_size,
        hidden_size=config.hidden_size,
        cell_type=config.cell_type,
        num_layers=config.encoder_layers,
        dropout=config.dropout
    )
    
    decoder = DecoderRNN(
        output_size=output_size,
        embed_size=config.embed_size,
        hidden_size=config.hidden_size,
        cell_type=config.cell_type,
        num_layers=config.encoder_layers,
        dropout=config.dropout
    )
    
    model = Seq2Seq(encoder, decoder).to(device)
    
    return model


In [15]:
# Define function to train model with given hyperparameters
def train_model(config=None):
    with wandb.init(config=config):
        # Access all hyperparameter values through wandb.config
        config = wandb.config
        
        # Load data
        data_dir = '/kaggle/working/dakshina_dataset_v1.0/hi/lexicons/'
        train_data = load_data(os.path.join(data_dir, 'hi.translit.sampled.train.tsv'))
        val_data = load_data(os.path.join(data_dir, 'hi.translit.sampled.dev.tsv'))
        test_data = load_data(os.path.join(data_dir, 'hi.translit.sampled.test.tsv'))
        # Create vocabulary
        latin_to_idx, idx_to_latin, devanagari_to_idx, idx_to_devanagari = create_vocab(train_data + val_data)
        
        # Encode data
        X_train, y_train, max_latin_len, max_devanagari_len = encode_data(train_data, latin_to_idx, devanagari_to_idx)
        X_val, y_val, _, _ = encode_data(val_data, latin_to_idx, devanagari_to_idx, max_latin_len, max_devanagari_len)
        X_test, y_test, _, _ = encode_data(test_data, latin_to_idx, devanagari_to_idx, max_latin_len, max_devanagari_len)
        # Create datasets and dataloaders
        train_dataset = TransliterationDataset(X_train, y_train)
        val_dataset = TransliterationDataset(X_val, y_val)
        test_dataset = TransliterationDataset(X_test, y_test)
        train_iterator = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_iterator = DataLoader(val_dataset, batch_size=config.batch_size)
        test_iterator = DataLoader(test_dataset, batch_size=config.batch_size)
        
        # Initialize model
        input_size = len(latin_to_idx)
        output_size = len(devanagari_to_idx)
        
        model = init_model(config, input_size, output_size)
        
        # Define optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        criterion = nn.CrossEntropyLoss(ignore_index=devanagari_to_idx['<pad>'])
        
        # Calculate model complexity
        complexity = calculate_model_complexity(
            m=config.embed_size,
            k=config.hidden_size,
            T=max(max_latin_len, max_devanagari_len),
            V=max(input_size, output_size)
        )
        
        wandb.log({
            'embedding_params': complexity['embedding_params'],
            'encoder_rnn_params': complexity['encoder_rnn_params'],
            'decoder_rnn_params': complexity['decoder_rnn_params'],
            'output_params': complexity['output_params'],
            'total_params': complexity['total_params'],
            'encoder_comp': complexity['encoder_comp'],
            'decoder_comp': complexity['decoder_comp'],
            'total_comp': complexity['total_comp']
        })
        print('embedding_params:', complexity['embedding_params'])
        print('encoder_rnn_params:', complexity['encoder_rnn_params'])
        print('decoder_rnn_params:', complexity['decoder_rnn_params'])
        print('output_params:', complexity['output_params'])
        print('total_params:', complexity['total_params'])
        print('encoder_comp:', complexity['encoder_comp'])
        print('decoder_comp:', complexity['decoder_comp'])
        print('total_comp:', complexity['total_comp'])
        
        # Training loop
        best_val_loss = float('inf')
        
        for epoch in range(config.epochs):
            start_time = time.time()
            
            train_loss = train(model, train_iterator, optimizer, criterion, config.clip, config.teacher_forcing_ratio)
            val_loss = evaluate(model, val_iterator, criterion)
            
            end_time = time.time()
            epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
            
            # Calculate accuracy
            accuracy, _ = calculate_accuracy(model, val_iterator, devanagari_to_idx, idx_to_devanagari)
            
            # Log metrics to wandb
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'accuracy': accuracy,
                'time': end_time - start_time
            })
            
            print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
            print(f'\tTrain Loss: {train_loss:.3f}')
            print(f'\tVal Loss: {val_loss:.3f}')
            print(f'\tAccuracy: {accuracy:.4f}')
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best-model.pt')
        
        # Final evaluation
        model.load_state_dict(torch.load('best-model.pt'))
        final_accuracy, predictions = calculate_accuracy(model, val_iterator, devanagari_to_idx, idx_to_devanagari)
        #final_accuracy, predictions = calculate_accuracy(model, val_iterator, devanagari_to_idx, idx_to_devanagari)
        test_accuracy, predictions = calculate_accuracy(model, test_iterator, devanagari_to_idx, idx_to_devanagari)
        wandb.log({'final_accuracy': final_accuracy})
        wandb.log({'final_accuracy_test': test_accuracy})
        print(f'\tfinal_accuracy: {final_accuracy:.4f}')
        print(f'\tfinal_accuracy_test: {test_accuracy:.4f}')
        
        # Log sample predictions
        samples = []
        for i, (pred, target) in enumerate(predictions[:10]):
            samples.append([i, pred, target, pred == target])
        
        wandb.log({
            "prediction_samples": wandb.Table(
                columns=["Index", "Prediction", "Target", "Correct"],
                data=samples
            )
        })
        print(samples)
        
        return final_accuracy

In [16]:
# Define sweep configuration
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'cell_type': {
            'values': ['RNN', 'LSTM', 'GRU']
        },
        'embed_size': {
            'values': [64, 128, 256]
        },
        'hidden_size': {
            'values': [128, 256, 512,1024]
        },
        'encoder_layers': {
            'values': [1, 2,3]
        },
        'dropout': {
            'values': [0.1, 0.2, 0.3]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'learning_rate': {
            'values': [0.001, 0.0005, 0.0001]
        },
        'teacher_forcing_ratio': {
            'values': [0.3, 0.5, 0.7]
        },
        'clip': {
            'values': [1.0, 5.0]
        },
        'epochs': {
            'value': 10
        }
    }
}


In [17]:
# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="final_no_attention_2")


Create sweep with ID: 18qajdbz
Sweep URL: https://wandb.ai/shivam-da24m018-iitmaana/final_no_attention_2/sweeps/18qajdbz


In [18]:
# Run sweep
wandb.agent(sweep_id, train_model, count=35)  # Run 20 experiments


wandb: Agent Starting Run: xn6qmd86 with config:
wandb: 	batch_size: 32
wandb: 	cell_type: GRU
wandb: 	clip: 1
wandb: 	dropout: 0.2
wandb: 	embed_size: 64
wandb: 	encoder_layers: 2
wandb: 	epochs: 10
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.001
wandb: 	teacher_forcing_ratio: 0.3
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250520_114243-xn6qmd86
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deep-sweep-1
wandb: ⭐️ View project at https://wandb.ai/shivam-da24m018-iitmaana/final_no_attention_2
wandb: 🧹 View sweep at https://wandb.ai/shivam-da24m018-iitmaana/final_no_attention_2/sweeps/18qajdbz
wandb: 🚀 View run at https://wandb.ai/shivam-da24m018-iitmaana/final_no_attention_2/runs/xn6qmd86


Sample Devanagari chars: ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ']
Sample Latin chars: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
embedding_params: 8448
encoder_rnn_params: 295424
decoder_rnn_params: 295424
output_params: 33858
total_params: 633154
encoder_comp: 6488064
decoder_comp: 7231488
total_comp: 13719552
Epoch: 01 | Time: 0.0m 53.67s
	Train Loss: 1.376
	Val Loss: 1.060
	Accuracy: 0.2767
Epoch: 02 | Time: 0.0m 52.71s
	Train Loss: 0.774
	Val Loss: 0.997
	Accuracy: 0.3272
Epoch: 03 | Time: 0.0m 52.94s
	Train Loss: 0.629
	Val Loss: 1.013
	Accuracy: 0.3373
Epoch: 04 | Time: 0.0m 53.05s
	Train Loss: 0.542
	Val Loss: 0.997
	Accuracy: 0.3621
Epoch: 05 | Time: 0.0m 52.90s
	Train Loss: 0.478
	Val Loss: 1.023
	Accuracy: 0.3692
Epoch: 06 | Time: 0.0m 52.86s
	Train Loss: 0.441
	Val Loss: 1.061
	Accuracy: 0.3678
Epoch: 07 | Time: 0.0m 53.25s
	Train Loss: 0.409
	Val Loss: 1.093
	Accuracy: 0.3607
Epoch: 08 | Time: 0.0m 53.35s
	Train Loss: 0.390
	Val Loss: 1.089
	Accuracy: 0.35

wandb: uploading artifact run-xn6qmd86-prediction_samples; uploading output.log; uploading config.yaml
wandb: uploading artifact run-xn6qmd86-prediction_samples
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            accuracy ▁▅▆▇██▇▇▇▇
wandb:        decoder_comp ▁
wandb:  decoder_rnn_params ▁
wandb:    embedding_params ▁
wandb:        encoder_comp ▁
wandb:  encoder_rnn_params ▁
wandb:               epoch ▁▂▃▃▄▅▆▆▇█
wandb:      final_accuracy ▁
wandb: final_accuracy_test ▁
wandb:       output_params ▁
wandb:                time █▁▃▄▂▂▅▆▅▆
wandb:          total_comp ▁
wandb:        total_params ▁
wandb:          train_loss █▄▃▂▂▂▁▁▁▁
wandb:            val_loss ▅▁▂▁▃▅▇▇█▇
wandb: 
wandb: Run summary:
wandb:            accuracy 0.35246
wandb:        decoder_comp 7231488
wandb:  decoder_rnn_params 295424
wandb:    embedding_params 8448
wandb:        encoder_comp 6488064
wandb:  encoder_rnn_params 295424
wandb:     

Sample Devanagari chars: ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ']
Sample Latin chars: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
embedding_params: 16896
encoder_rnn_params: 1180672
decoder_rnn_params: 1180672
output_params: 67650
total_params: 2445890
encoder_comp: 25952256
decoder_comp: 27439104
total_comp: 53391360
Epoch: 01 | Time: 0.0m 36.18s
	Train Loss: 3.018
	Val Loss: 3.355
	Accuracy: 0.0000
Epoch: 02 | Time: 0.0m 35.96s
	Train Loss: 2.904
	Val Loss: 3.248
	Accuracy: 0.0000
Epoch: 03 | Time: 0.0m 35.71s
	Train Loss: 2.866
	Val Loss: 3.405
	Accuracy: 0.0005
Epoch: 04 | Time: 0.0m 35.81s
	Train Loss: 2.843
	Val Loss: 3.425
	Accuracy: 0.0000
Epoch: 05 | Time: 0.0m 35.82s
	Train Loss: 2.829
	Val Loss: 3.339
	Accuracy: 0.0000
Epoch: 06 | Time: 0.0m 35.83s
	Train Loss: 2.816
	Val Loss: 3.420
	Accuracy: 0.0000
Epoch: 07 | Time: 0.0m 35.78s
	Train Loss: 2.798
	Val Loss: 3.295
	Accuracy: 0.0000
Epoch: 08 | Time: 0.0m 35.86s
	Train Loss: 2.801
	Val Loss: 3.347
	Accuracy

wandb: uploading artifact run-gxq2ns8k-prediction_samples; uploading media/table/prediction_samples_13_a84fe7dc34c8eb5fb9ce.table.json; uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading artifact run-gxq2ns8k-prediction_samples
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            accuracy ▁▁█▁▁▁▁▁▁▁
wandb:        decoder_comp ▁
wandb:  decoder_rnn_params ▁
wandb:    embedding_params ▁
wandb:        encoder_comp ▁
wandb:  encoder_rnn_params ▁
wandb:               epoch ▁▂▃▃▄▅▆▆▇█
wandb:      final_accuracy ▁
wandb: final_accuracy_test ▁
wandb:       output_params ▁
wandb:                time █▅▁▃▃▃▂▃▃▂
wandb:          total_comp ▁
wandb:        total_params ▁
wandb:          train_loss █▄▃▂▂▂▁▁▁▁
wandb:            val_loss ▅▁▇█▅█▃▅▂▅
wandb: 
wandb: Run summary:
wandb:            accuracy 0
wandb:        decoder_comp 27439104
wandb:  decoder_rnn_params 1180672
wandb:   

...

wandb: uploading artifact run-zqmer835-prediction_samples; uploading media/table/prediction_samples_13_b0c6549a5d4369e05396.table.json; uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading artifact run-zqmer835-prediction_samples
wandb: uploading history steps 11-13, summary, console lines 50-52
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            accuracy ▁▅▆▇▇█▇▇▇▇
wandb:        decoder_comp ▁
wandb:  decoder_rnn_params ▁
wandb:    embedding_params ▁
wandb:        encoder_comp ▁
wandb:  encoder_rnn_params ▁
wandb:               epoch ▁▂▃▃▄▅▆▆▇█
wandb:      final_accuracy ▁
wandb: final_accuracy_test ▁
wandb:       output_params ▁
wandb:                time ▁▃▆▆▃▁▃▇█▆
wandb:          total_comp ▁
wandb:        total_params ▁
wandb:          train_loss █▄▃▂▂▂▁▁▁▁
wandb:            val_loss █▃▁▁▁▁▂▂▂▄
wandb: 
wandb: Run summary:
wandb:            accuracy 0.35154
wandb: 

Sample Devanagari chars: ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ']
Sample Latin chars: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
embedding_params: 33792
encoder_rnn_params: 131328
decoder_rnn_params: 131328
output_params: 16962
total_params: 313410
encoder_comp: 2883584
decoder_comp: 3255296
total_comp: 6138880
Epoch: 01 | Time: 1.0m 8.61s
	Train Loss: 2.604
	Val Loss: 2.246
	Accuracy: 0.0255
Epoch: 02 | Time: 1.0m 7.91s
	Train Loss: 1.240
	Val Loss: 1.482
	Accuracy: 0.2396
Epoch: 03 | Time: 1.0m 8.11s
	Train Loss: 0.774
	Val Loss: 1.270
	Accuracy: 0.2958
Epoch: 04 | Time: 1.0m 8.20s
	Train Loss: 0.608
	Val Loss: 1.219
	Accuracy: 0.3502
Epoch: 05 | Time: 1.0m 8.24s
	Train Loss: 0.516
	Val Loss: 1.246
	Accuracy: 0.3547
Epoch: 06 | Time: 1.0m 8.18s
	Train Loss: 0.444
	Val Loss: 1.226
	Accuracy: 0.3731
Epoch: 07 | Time: 1.0m 8.23s
	Train Loss: 0.391
	Val Loss: 1.243
	Accuracy: 0.3853
Epoch: 08 | Time: 1.0m 8.41s
	Train Loss: 0.344
	Val Loss: 1.259
	Accuracy: 0.3981
Epoch

wandb: uploading artifact run-c9okovjb-prediction_samples; uploading media/table/prediction_samples_13_534bd0b503c233036062.table.json; uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading artifact run-c9okovjb-prediction_samples
wandb: 
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            accuracy ▁▅▆▇▇█████
wandb:        decoder_comp ▁
wandb:  decoder_rnn_params ▁
wandb:    embedding_params ▁
wandb:        encoder_comp ▁
wandb:  encoder_rnn_params ▁
wandb:               epoch ▁▂▃▃▄▅▆▆▇█
wandb:      final_accuracy ▁
wandb: final_accuracy_test ▁
wandb:       output_params ▁
wandb:                time █▁▃▄▄▄▄▆▆▁
wandb:          total_comp ▁
wandb:        total_params ▁
wandb:          train_loss █▄▃▂▂▂▁▁▁▁
wandb:            val_loss █▃▁▁▁▁▁▁▁▂
wandb: 
wandb: Run summary:
wandb:            accuracy 0.39468
wandb:        decoder_comp 3255296
wandb:  decoder_rnn_params 1313

Sample Devanagari chars: ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ']
Sample Latin chars: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
embedding_params: 8448
encoder_rnn_params: 295424
decoder_rnn_params: 295424
output_params: 33858
total_params: 633154
encoder_comp: 6488064
decoder_comp: 7231488
total_comp: 13719552
Epoch: 01 | Time: 0.0m 14.27s
	Train Loss: 2.965
	Val Loss: 3.220
	Accuracy: 0.0000
Epoch: 02 | Time: 0.0m 14.05s
	Train Loss: 2.831
	Val Loss: 3.859
	Accuracy: 0.0000
Epoch: 03 | Time: 0.0m 14.23s
	Train Loss: 2.784
	Val Loss: 3.276
	Accuracy: 0.0000
Epoch: 04 | Time: 0.0m 14.06s
	Train Loss: 2.771
	Val Loss: 3.742
	Accuracy: 0.0005
Epoch: 05 | Time: 0.0m 14.18s
	Train Loss: 2.747
	Val Loss: 3.225
	Accuracy: 0.0000
Epoch: 06 | Time: 0.0m 14.04s
	Train Loss: 2.730
	Val Loss: 3.358
	Accuracy: 0.0000
Epoch: 07 | Time: 0.0m 14.36s
	Train Loss: 2.706
	Val Loss: 3.424
	Accuracy: 0.0000
Epoch: 08 | Time: 0.0m 14.13s
	Train Loss: 2.704
	Val Loss: 3.444
	Accuracy: 0.00

wandb: uploading artifact run-b4addmm3-prediction_samples; uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading artifact run-b4addmm3-prediction_samples
wandb: uploading history steps 10-13, summary, console lines 46-52
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            accuracy ▁▁▁▃▁▁▁▃█▁
wandb:        decoder_comp ▁
wandb:  decoder_rnn_params ▁
wandb:    embedding_params ▁
wandb:        encoder_comp ▁
wandb:  encoder_rnn_params ▁
wandb:               epoch ▁▂▃▃▄▅▆▆▇█
wandb:      final_accuracy ▁
wandb: final_accuracy_test ▁
wandb:       output_params ▁
wandb:                time ▇▅▇▅▆▅█▆▇▁
wandb:          total_comp ▁
wandb:        total_params ▁
wandb:          train_loss █▅▃▃▂▂▁▁▁▂
wandb:            val_loss ▁█▂▇▁▃▃▃▁▂
wandb: 
wandb: Run summary:
wandb:            accuracy 0
wandb:        decoder_comp 7231488
wandb:  decoder_rnn_params 295424
wandb:    embedding_p